In [62]:
import numpy as np
import cv2
import dlib
from imutils import face_utils
import glob
from random import shuffle
from sklearn.model_selection import train_test_split
from sklearn import svm, metrics

### Load the pretrained Dlib model for face landmarks

In [12]:
# Directory of the pretrained model
p = "shape_predictor_68_face_landmarks.dat"

# Pretrained dlib model
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(p)

# Change hardcoded size
size = (480, 640, 3)

### Define the 3D points in World Coordinates. These will be the same for all images

In [13]:
# 3D model points.
model_points = np.array([
                            (0.0, 0.0, 0.0),             # Nose tip
                            (0.0, -330.0, -65.0),        # Chin
                            (-225.0, 170.0, -135.0),     # Left eye left corner
                            (225.0, 170.0, -135.0),      # Right eye right corne
                            (-150.0, -150.0, -125.0),    # Left Mouth corner
                            (150.0, -150.0, -125.0)      # Right mouth corner
                         
                        ])

### Approximate Camera Intrinsic Parameters

In [14]:
# Approximate camera intrinsic parameters
focal_length = size[1]
center = (size[1]/2, size[0]/2)
camera_matrix = np.array(
                         [[focal_length, 0, center[0]],
                         [0, focal_length, center[1]],
                         [0, 0, 1]], dtype = "double"
                         )
dist_coeffs = np.zeros((4,1)) # Assuming no lens distortion

### Load the dataset

In [31]:
files_positive = glob.glob('test_dataset/positive/*.jpg')
files_negative = glob.glob('test_dataset/negative/*.jpg')

X = np.zeros((52, 6, 1))

for i in range(len(files_positive) + len(files_negative)):
    if i<len(files_positive):
        image = cv2.imread(files_positive[i])
    else:
        image = cv2.imread(files_negative[i-26])
    
    # Get faces into webcam's image
    rects = detector(image, 0)

    # Loop over every face detected. Here we only have on face each time
    for rect in rects:
        # Make the prediction and transfom it to numpy array
        shape = predictor(image, rect)
        landmarks = face_utils.shape_to_np(shape)
        
    # Grab the 2D coordinates of our six sample points
    image_points = np.array([
        landmarks[33, :],     # Nose tip
        landmarks[8, :],     # Chin
        landmarks[36, :],     # Left eye left corner
        landmarks[45, :],     # Right eye right corner
        landmarks[48, :],     # Left Mouth corner
        landmarks[54, :]      # Right mouth corner
    ], dtype="double")

    # Solve the PnP problem with the parameters specified above
    (success, rotation_vector, translation_vector) = cv2.solvePnP(
        model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_ITERATIVE
        )
#     print("Rotation Vector:\n {0}".format(rotation_vector))
#     print("Translation Vector:\n {0}".format(translation_vector))
    
    X[i, :] = np.concatenate((rotation_vector, translation_vector), axis=0)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
[[[-2.92169335e+00]
  [ 3.42095614e-02]
  [-3.62215831e-01]
  [ 1.62104510e+02]
  [ 8.87853615e+01]
  [ 2.17018419e+03]]

 [[-2.97798950e+00]
  [ 4.78704725e-02]
  [-3.29580993e-01]
  [ 1.42695668e+02]
  [ 5.19086824e+01]
  [ 2.17032161e+03]]

 [[-2.92621287e+00]
  [ 1.84997323e-02]
  [-3.70240819e-01]
  [ 1.71627464e+02]
  [ 8.96016943e+01]
  [ 2.13865282e+03]]

 [[-2.78627228e+00]
  [-2.86201073e-03]
  [-5.00350785e-01]
  [ 1.33310575e+02]
  [ 1.07916371e+02]
  [ 1.99295840e+03]]

 [[-2.91673205e+00]
  [ 2.25644603e-02]
  [-2.25882268e-01]
  [ 1.68212931e+02]
  [ 6.29700273e+01]
  [ 2.15535384e+03]]]


In [33]:
np.savetxt('input.out', X.squeeze(), delimiter=',')   # X is an array

In [36]:
X = X.squeeze()
print(X.shape)

(52, 6)


In [44]:
y = np.concatenate((np.zeros((26)), np.ones((26))), axis=0)
print(y.shape)

(52,)


In [45]:
np.savetxt('output.out', y, delimiter=',')   # X is an array

### Shuffle and split dataset

In [59]:
indexes = [i for i in range(52)]
shuffle(indexes)

X = X[indexes, :]
y = y[indexes, ]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [60]:
print(X_train.shape)

(36, 6)


In [63]:
classifier = svm.SVC(kernel='linear')

In [64]:
classifier.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [65]:
y_pred = classifier.predict(X_test)

In [66]:
print('Accuracy: ', metrics.accuracy_score(y_test, y_pred))

Accuracy:  0.8125
